## Modified introduction using forex data

This is the trading rule example shown in [the introduction](https://github.com/robcarver17/pysystemtrade/blob/master/docs/introduction.md) but modified to use Interactive Brokers instead of CSV files as data source.  

IB requires a minimum equity and a monthly subscription to provide historical data on future contracts.  This example was modified to use FX prices instead futures to make it runnable with free unfunded paper trading accounts.  Note that Rob [does not recommend trading FX spot data with IB due to their high fees](https://github.com/robcarver17/pysystemtrade/issues/517#issuecomment-1010770678).

First, import the required packages and initialize ib_insync.  

In [ ]:
import logging

logging.getLogger('base_system').setLevel(logging.WARNING)
logging.getLogger('config').setLevel(logging.WARNING)

In [ ]:
import functools
import itertools

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from arctic import Arctic
from sysdata.config.configdata import Config
from systems.provided.futures_chapter15.basesystem import futures_system
from sysdata.data_blob import dataBlob
from sysdata.sim.db_futures_sim_data import dbFuturesSimData
from systems.basesystem import System


def update_config(
    config,
    instrument_weights=None,
    notional_trading_capital=None,
    base_currency=None,
    start_date=None,
    buffer_size=None,
    forecast_div_multiplier=None,
    forecast_weights=None,
    percentage_vol_target=None,
):

    config.notional_trading_capital = notional_trading_capital or config.notional_trading_capital
    config.base_currency = base_currency or config.base_currency
    config.percentage_vol_target = percentage_vol_target or config.percentage_vol_target
    config.buffer_size = buffer_size or config.buffer_size
    config.start_date = start_date or config.start_date
    config.forecast_div_multiplier = forecast_div_multiplier or config.forecast_div_multiplier
    config.forecast_weights = forecast_weights or config.forecast_weights
    config.instrument_weights = instrument_weights or config.instrument_weights
    return config



def run_backtest(
    config_file: str="private.futures_system.config.yaml",
    data=None,
    config=None,
    **kwargs,
):
    data = data or dbFuturesSimData()

    config=config or Config(config_file)
    update_config(config, **kwargs)
    system = futures_system(config=config, data=data, trading_rules=)
    return system, system.accounts.portfolio()


def rolled_prices(lib, symbol):
    def get_prices(n):
        symbols = lib.list_symbols(regex=f'Day/{n}')
        return pd.concat(
            (lib.read(c).data[['FINAL', 'VOLUME']] for c in  symbols),
            axis=1,
            keys=symbols)
    def adjusted_prices(prices):
        return prices[
            (prices.loc[:,(slice(None), 'FINAL')].shift(-1, axis=1).fillna(0.0)
             <= prices.loc[:,(slice(None), 'FINAL')])
        ].loc[:,(slice(None), 'FINAL')].ffill(axis=1).iloc[:,-1].rename(symbol)
    return adjusted_prices(get_prices(symbol))


def for_each(
    axis,
    system,
    method,
    *args,
    funcs=(),
    ignore_zero_weighted=True,
    **kwargs,
):
    """
    :param axis: instrument or strategy
    :param system: the system
    :param method: function accepting instrument_code or trading rule name.
    :param funcs: a list of functions to apply to output of method
    :param ignore_zero_weighted: if true, exclude zero weighted instruments or rules
    """
    rule_variation_name = kwargs.get('rule_variation_name')
    instrument_code = kwargs.get('instrument_code')

    def _validate_args():
        if rule_variation_name and instrument_code:
            raise ValueError("provide only one of instrument_code and rule_variation_name")
        if axis == 'instrument' and instrument_code:
            raise ValueError(axis, instrument_code)
        if axis == 'strategy' and rule_variation_name:
            raise ValueError(axis, rule_variation_name)
        if axis == 'all' and (rule_variation_name or instrument_code):
            raise ValueError(axis, (instrument_code or rule_variation_name))

    _validate_args()

    def compz(t, funcs):
        return t if not funcs else compz(funcs[0](t), funcs[1:])

    instruments = system.data.get_all_instrument_data_as_df()
    instruments = instruments[instruments.index.isin(system.get_instrument_list())]
    
    strategy_ignore = ignore = lambda i: (system.config.forecast_weights.get(i['rule_variation_name'], 0) == 0.0
                        if ignore_zero_weighted else False)
    instrument_ignore = lambda i: (system.config.instrument_weights[i['instrument_code']] == 0.0
                        if ignore_zero_weighted else False)

    if axis=='instrument':
        ignore = instrument_ignore
        elems = [{'instrument_code': j, **kwargs} for j in system.config.instrument_weights]
    elif axis=='strategy':
        ignore = strategy_ignore
        elems = [{'rule_variation_name': j, **kwargs} for j in system.config.forecast_weights]
    elif axis=='all':
        elems = [{'instrument_code': i, 'rule_variation_name': s, **kwargs}
                 for i, s in itertools.product(
                    system.get_instrument_list(),
                    system.rules.trading_rules().keys(),
        )]
        ignore = lambda i: (instrument_ignore(i['instrument_code'])
                                or strategy_ignore(i['rule_variation_name']))
    else:
        raise ValueError(f'One of strategy or instrument, not {axis}.')

    key = (('rule_variation_name' if axis=='strategy' else 'instrument_code')
           if axis!='all'
           else ('instrument_code', 'rule_variation_name'))

    if isinstance(key, tuple):
        keys = [(i['instrument_code'], i['rule_variation_name']) for i in elems if not ignore(i)]
    else:
        keys = [i[key] for i in elems if not ignore(i)]

    return pd.concat(
        (compz(method(**i), funcs)
         for i in elems if not ignore(i)),
            axis=1,
            keys=keys,
        )

In [ ]:
config = Config('private.futures_system.config.yaml')
config.forecast_weights

In [ ]:
instrument_weights = {
 'CAD_micro': 0.1,
 'COPPER-micro': 0.1,
 'CORN': 0.1,
 'EDOLLAR': 0.1,
 'EUROSTX': 0.1,
 'NASDAQ_micro': 0.1,
 'US10': 0,
 'SP500_micro': 0.1,
 'V2X': 0.1,
 'MXP': 0.1,
 'GASOILINE_micro': 0.1,
}

forecast_weights = {
 'ewmac16_64': 0.21,
 'ewmac32_128': 0.31,
 'ewmac64_256': 0.41,
}

data = dbFuturesSimData()

config = Config('private.futures_system.config.yaml')
config.use_instrument_weight_estimates = True
system, pf = run_backtest(
    notional_trading_capital=70_000,
    percentage_vol_target=25,
    start_date='1985-01-01',
    data=data,
    forecast_weights=forecast_weights,
    instrument_weights=instrument_weights,
    buffer_size=.85,
    config=config,
)

In [ ]:
fig, ax = plt.subplots()
for_each('strategy', system,
         system.rules.get_raw_forecast,
         instrument_code='SP500_micro',
        ).plot(ax=ax)

fig, ax = plt.subplots()
for_each('instrument', system,
         system.accounts.get_buffered_position,
        ).plot(ax=ax)


fig, ax = plt.subplots()
for_each('instrument',
         system, system.accounts.pandl_for_instrument,
         funcs=[pd.DataFrame.cumsum],).plot(ax=ax)

fig, ax = plt.subplots()
pf.curve().plot(ax=ax)

fig, ax = plt.subplots()
vols = for_each('instrument', system, system.rawdata.get_daily_percentage_volatility)
vols.ewm(128).mean().plot(ax=ax)


fig, ax = plt.subplots()
account = pf.percent.curve()
sp = system.accounts.get_daily_prices('SP500_micro')
sprets = (1+sp.pct_change()).cumprod()
sprets.rename('S&P').plot(ax=ax)
(1+account/100).rename('account').plot(ax=ax)
fig.legend()
# (1+sp_account/100).plot()
#plt.plot(sp.index, 1+sp_account/100, label='SP Account')
pf.percent.stats()

In [ ]:
from sysproduction.data.backtest import dataBacktest
from sysproduction.strategy_code import report_system_classic

bt = dataBacktest().get_most_recent_backtest(strategy_name='example')

#report_system_classic.report_system_classic(data, bt)

In [ ]:
bt.system.accounts.portfolio().curve().plot()

In [ ]:
data = for_each('instrument', system, system.accounts.get_buffered_position).diff()
pd.concat((data.SP500_micro[data.SP500_micro.ne(0)].dropna().index.to_series(),
           np.sign(data.SP500_micro[data.SP500_micro.ne(0)].dropna())), axis=1)
           

In [ ]:
# system.accounts.get_buffered_position('SP500_micro', 'meanreversion').plot()
system.accounts.portfolio().percent.curve().plot()
system.accounts.portfolio().percent.stats()
#system.accounts.get_buffered_position('SP500_micro').plot()

In [ ]:
system.rules

In [ ]:

fig,ax = plt.subplots()
ax.xaxis.set_major_locator(matplotlib.dates.YearLocator(base=1))
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))


#pf.percent.curve().plot(ax=ax)
buffered_pos = system.accounts.get_actual_buffers_for_position('SP500_micro')

actual_pos = system.accounts.get_actual_position('SP500_micro')
actual_pos.plot(ax=ax)
buffered_pos.plot(ax=ax)
#system.accounts.get_capped_forecast('EDOLLAR', 'ewmac64_256').rename('ewmac64_256').tail(120).plot()
#system.accounts.get_capped_forecast('EDOLLAR', 'ewmac4_16').rename('ewmac4_16').tail(120).plot()


In [ ]:
system.accounts.get_buffered_position('V2X').plot()

In [ ]:
(1+sp.pct_change()).cumprod().plot()

In [ ]:
system.accounts.get_daily_returns_volatility('SP500')

In [ ]:

import numpy as np
#system.combForecast.get_monthly_raw_forecast_weights_estimated('SP500_micro')#.diff().ewm(120).std(),
# np.corrcoef(
#     system.accounts.get_daily_returns_volatility('SP500_micro'),
#     system.combForecast.get_monthly_raw_forecast_weights_estimated('SP500_micro')#.diff().ewm(120).std(),
# )
plot = ((system.accounts.get_buffered_position('SP500_micro')
 * 5
 * system.accounts.get_daily_prices('SP500_micro'))
    .plot())


In [ ]:
data.get_all_instrument_data_as_df()

In [ ]:
instrument_code = 'SP500_micro'
pv = data.get_all_instrument_data_as_df()['Pointsize'].loc['SP500_micro']
position = system.accounts.get_buffered_position(instrument_code)
price = system.accounts.get_daily_prices(instrument_code)

position.plot(secondary_y=True, grid=True)
price.plot(grid=True)

In [ ]:
stats, *_ = system.accounts.portfolio().stats()
pd.DataFrame(stats).set_index(0).transpose().rename({1: 'example'}, axis=0)

In [ ]:
df = pd.concat((adjusted_prices.read(c).data.squeeze().rename(c) for c in adjusted_prices.list_symbols()), axis=1)

In [ ]:
pip install --upgrade nuitka 

In [ ]:
fig, ax = plt.subplots()
#pf.percent.curve().rename('example').plot(ax=ax)
pxs = adjusted_prices.read('SP500').data
ax.plot(pxs.index, pxs, label='SP500')
fig.legend()
pxs.tail()




In [ ]:
from sysproduction.data import backtest

In [ ]:
bt = backtest.dataBacktest()

In [ ]:
bt = bt.get_most_recent_backtest('example')

In [ ]:
adjusted_prices.read('EDOLLAR').data.tail(50)

In [ ]:
system.portfolio.